In [ ]:
# default_exp 
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import FRED
if torch.__version__[:4] == '1.13': # If using pytorch with MPS, use Apple silicon GPU acceleration
    device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.has_mps else "cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)
%load_ext autoreload
%autoreload 2

Using device mps


# 05b Toy Dataset Benchmarker
> Run a given model on all of our toy datasets (optionally, do so many times), and produce training gifs and loss charts for each.

In [ ]:
%load_ext snakeviz

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [ ]:
# Code to test visualization
import random
# Simple function to print messages 
def print_msg():
    for i in range(10):
        print("Program completed")

# Generate random data
def generate():
    data = [random.randint(0, 99) for p in range(0, 1000)]
    return data

# Function to search 
def search_function(data):
    for i in data:
        if i in [100,200,300,400,500]:
            print("success")

def main():
    data=generate()
    search_function(data)
    print_msg()

%prun -D program.prof main() 

Program completed
Program completed
Program completed
Program completed
Program completed
Program completed
Program completed
Program completed
Program completed
Program completed
 
*** Profile stats marshalled to file 'program.prof'.


         5493 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 socket.py:543(send)
     1000    0.000    0.000    0.001    0.000 random.py:290(randrange)
     1000    0.000    0.000    0.000    0.000 random.py:237(_randbelow_with_getrandbits)
     1000    0.000    0.000    0.001    0.000 random.py:334(randint)
        1    0.000    0.000    0.001    0.001 1419858537.py:10(<listcomp>)
        1    0.000    0.000    0.000    0.000 1419858537.py:14(search_function)
     1307    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
     1000    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
       20    0.000    0.000    0.001    0.000 iostream.py:525(write)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
       10    0.000    0.000    0.001    0.000 {built-in method builtins.